In [1]:
from textgenrnn import textgenrnn
import syllapy

/opt/conda/lib/python3.8/site-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


In [2]:
download_fn = 'poetry.ndjson.gz'
! rm $download_fn
! curl -o $download_fn http://static.decontextualize.com/gutenberg-poetry-v001.ndjson.gz

rm: cannot remove 'poetry.ndjson.gz': No such file or directory
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 52.2M  100 52.2M    0     0  32.8M      0  0:00:01  0:00:01 --:--:-- 32.8M


In [3]:
poetry_fn = download_fn.replace('.gz', '')
! rm $poetry_fn
! gzip -d $download_fn

rm: cannot remove 'poetry.ndjson': No such file or directory


In [4]:
with open('poetry.ndjson', 'r') as infile, open('poetry.txt', 'w') as outfile:
    for line in infile:
        nice_text = line.replace('{"s": "', '').replace('", "gid": "19"}', '')
        outfile.write(nice_text)
! head -1200 poetry.txt > poetry_short.txt

In [5]:
textgen = textgenrnn()
textgen.train_from_file('poetry_short.txt', num_epochs=1)

1,200 texts collected.
Training on 40,645 character sequences.
317/317 [==============================] - 48s 141ms/step - loss: 1.6087
####################
Temperature: 0.2
####################
Showed the should the shower,

Showed the fell of the liver,

And the water of the should his wind,

####################
Temperature: 0.5
####################
And his waited and burthes and should mean leaf in the spawn;

They heart his straighter that the moon;

From the bead of the moon,

####################
Temperature: 1.0
####################
Full his curres by upon,

Old your his streak you have every vaghies?

lasty, a moon with his lending,



In [ ]:
a = textgen.generate(100, temperature=0.5, return_as_list=True)

 67%|██████▋   | 67/100 [01:57<00:53,  1.62s/it]

In [ ]:
l_filtered = []
for l in a:
    length = syllapy.count(l)
    if 8 <= length <= 12:
        l_filtered.append(l)

In [ ]:
class Poem:
    def __init__(self, lines):
        '''https://en.wikipedia.org/wiki/Sestina'''
        assert isinstance(lines, list)
        assert len(lines) >= 39
        
        self.end_words = {i:'' for i in range(6)}
        
        # I subtract 1 from each word index to be consistent with python's
        # zero-based indexing, but I start with the 1-based to be consistent with 
        # wikipedia documentation
        self.rotation = [6, 1, 5, 2, 4, 3, \
                         3, 6, 4, 1, 2, 5, \
                         5, 3, 2, 6, 1, 4, \
                         4, 5, 1, 3, 6, 2, \
                         2, 4, 6, 5, 3, 1]
        self.rotation = [x - 1 for x in self.rotation]
        self.envoi = [(6, 2), (1, 4), (5, 3)]
        self.envoi = [(t[0]-1, t[1]-1) for t in self.envoi]
        
        self.sestina = []
        self.get_end_words(lines[:6])
        self.replace_last_words(lines[6:36])
        self.compose_envoi(lines[36:39])
    
    def get_end_words(self, first_6_lines):
        assert len(first_6_lines) == 6
        for i, line in enumerate(first_6_lines):
            self.end_words[i] = line.split()[-1]
        self.sestina.extend(first_6_lines)
    
    def replace_last_words(self, middle_lines):
        assert len(middle_lines) == 30
        for i, line in enumerate(middle_lines):
            word_i = self.rotation[i]
            new_line = line.split()[:-1] + [self.end_words[word_i]]
            self.sestina.append(' '.join(new_line))
    
    def compose_envoi(self, last_lines):
        assert len(last_lines) == 3
        for i, line in enumerate(last_lines):
            pts = line.split()[:-1]
            half = len(pts) // 2
            etpl = self.envoi[i]
            new_line = pts[:(half-1)] + [self.end_words[etpl[0]]] + \
                       pts[half:-1] + [self.end_words[etpl[1]]]
            self.sestina.append(' '.join(new_line))
    
    def recite(self):
        for i, line in enumerate(self.sestina):
            print(line)
            if (i+1)%6 == 0 and i > 0:
                print()


In [ ]:
p = Poem(l_filtered)
p.recite()